In [117]:
%load_ext autoreload
%autoreload 2

from excel_matches_livingscores import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. load matches

In [88]:
with open('matches_excel.json') as f:
    matches = json.load(f)
print(len(matches))

91035


# 2. Load Booking data

In [89]:
print("Loading Booking hotels")
bkgh = load_booking(matches.values())
print(len(bkgh))

Loading Booking hotels
91035


# 2.b Load Amadeus spreadsheet

In [90]:
print("Loading Amadeus hotels")
amdh_full = pd.read_excel('./Amadeus All Properties - FEB 2017 17022017 .xlsx', header=0, index_col='PROPERTY_CODE', skiprows=1)
amdh = convert_amd_df_to_matching_format(amdh_full)
amdh.amd_id = amdh.amd_id.values.astype(str)
def pc_to_bkg(pc):
    return matches.get(pc)
amdh['bkg_id'] = amdh['amd_id'].apply(pc_to_bkg)

namdh = len(amdh)
print("Loaded %d hotels" % namdh)

Loading Amadeus hotels
Loaded 140709 hotels


# 3. Inspect chain matches

In [95]:
# Don't compute them again: chain_matches_excel.json has been edited with manually checked matches
get_chain_matches(amdh, bkgh, matches)

{u'AC HOTELES': defaultdict(int,
             {u'AC Hotels by Marriott': 50,
              u'Aloft': 1,
              u'Blue Bay Resorts': 1,
              u'Copthorne Hotels': 1,
              u'Courtyard by Marriott': 1,
              u'Hyatt Place': 1,
              u'Hyatt house': 1,
              u'JW Marriott Hotels & Resorts': 2,
              u'Marriott Hotels & Resorts': 1,
              u'NH Hotels': 2,
              'NONE': 8,
              u'Residence Inn': 2,
              u'SB Hotels': 1,
              u'VP Hotels': 1,
              u'Westin': 1}),
 u'ACCOR HOTELS': defaultdict(int,
             {u'Adagio Access Aparthotels': 38,
              u'Adagio Aparthotels': 24,
              u'Agon Group, ibis Styles': 1,
              u'B&amp;B Hotels': 5,
              u'Best Western': 1,
              u'Caesar Hotels': 1,
              u'Campanile': 2,
              u'Cerise Hotels &amp; Residences': 1,
              u'City Express Hotels': 1,
              u'Courtyard by Marr

In [92]:
amdh.bkg_id = amdh.amd_id.apply(lambda i: matches[i] if i in matches else None)
matches_df = amdh.join(bkgh, on='bkg_id', how='inner', rsuffix='_bkg')
matches_df = matches_df[['amd_id', 'bkg_id_bkg',u'name', 'name_bkg', u'chain', 'chain_bkg', 'lat',  u'lng', u'lat_bkg', u'lng_bkg']]
mdf = matches_df
mdf.chain_bkg.fillna('', inplace=True)
len(matches_df)

91035

In [ ]:
# Explore suspicious chain matches ( Example )
chain_a = "RESIDENCE INN HOTELS"
chain_b = "Marriott Hotels & Resorts"

filter_a = (mdf.chain == chain_a) if chain_a else (mdf.chain.isnull())
filter_b = (mdf.chain_bkg == chain_b) if chain_b else (mdf.chain_bkg.isnull())
mdf[filter_a & filter_b]

# 4. clean up erroneous chain matches

In [93]:
chain_matches = load_commented_json('chain_matches_excel_suspicious.json')

In [94]:
sc = mdf.copy() # candidates to suspicious

sc.name = sc.name.apply(normalize)
sc.name_bkg = sc.name_bkg.apply(normalize)
sc.chain = sc.chain.astype(unicode).apply(normalize)
sc.chain_bkg = sc.chain_bkg.astype(unicode).apply(normalize)

filter_neq_name = sc.name != sc.name_bkg
sc = sc[filter_neq_name]

len(sc)

47652

In [97]:
suspicious = pd.DataFrame(data=[],columns=mdf.columns)

for chain_a, d in chain_matches.items():
    for chain_b in d:
        chain_a = normalize(unicode(chain_a))
        chain_b = normalize(unicode(chain_b))
        filter_a = (sc.chain == chain_a) if chain_a != 'none' else (sc.chain.isnull())
        filter_b = (sc.chain_bkg == chain_b) if chain_b != 'none' else (sc.chain_bkg.isnull())
        if chain_a:
            filter_b = filter_b & ~(sc.name_bkg.str.contains(chain_a))
        if chain_b:
            filter_a = filter_a & ~(sc.name.str.contains(chain_b))
            
        suspicious = suspicious.append(sc[filter_a & filter_b])

len(suspicious)

41464

In [103]:
def _get_geo_dist(lat1, lng1, lat2, lng2):
    """
        Geographic distance using Haversine formula
    """
    r = 6371000  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    lambda1 = math.radians(lng1)
    lambda2 = math.radians(lng2)
    haversin = lambda theta: (1 - math.cos(theta)) / 2.0
    h = haversin(phi2 - phi1) + math.cos(phi1) * math.cos(phi2) * \
        haversin(lambda2 - lambda1)
    c = 2 * math.atan2(math.sqrt(h), math.sqrt(1 - h))
    
    return r * c

vect_geo_dist = np.vectorize(_get_geo_dist)

def row_name_sim(row):
    return get_name_sim(row['name'], row['name_bkg'])

def row_name_sim_sw(row):
    return get_name_sim(row['name'], row['name_bkg'], swap_words=True)

def row_chain_sim(row):
    return get_name_sim(row['chain'], row['chain_bkg'])

def row_chain_sim_sw(row):
    return get_name_sim(row['chain'], row['chain_bkg'], swap_words=True)

In [104]:
s = suspicious

In [105]:
s['dist'] = vect_geo_dist(s.lat, s.lng, s.lat_bkg, s.lng_bkg)

In [118]:
s['name_sim'] = s.apply(row_name_sim, axis=1)

In [121]:
s['name_sim_sw'] = s.apply(row_name_sim_sw, axis=1)

In [122]:
s['chain_sim'] = s.apply(row_chain_sim, axis=1)

In [123]:
s['chain_sim_sw'] = s.apply(row_chain_sim_sw, axis=1)

In [129]:
def row_name_included(row):
    n = normalize(row['name']).replace(' ', '')    
    nb = normalize(row['name_bkg']).replace(' ', '')
    return (n in nb) or (nb in n)

def row_chain_included(row):
    n = normalize(row['chain']).replace(' ', '')    
    nb = normalize(row['chain_bkg']).replace(' ', '')
    return (n and n in nb) or (nb and nb in n)

In [130]:
s['name_included'] = s.apply(row_name_included, axis=1)

In [131]:
s['chain_included'] = s.apply(row_chain_included, axis=1)

In [133]:
s.sample(50)

,amd_id,bkg_id_bkg,name,name_bkg,chain,chain_bkg,lat,lng,lat_bkg,lng_bkg,dist,name_sim,name_sim_sw,chain_sim,chain_sim_sw,name_included,chain_included
HSXMNAFX,700206284.0,1548038.0,add inn xiamen,qininn xiamen gucuo,hrs gds,,24.4259,118.12358,24.426950,118.123949,122.569637,0.606061,0.666667,0.000000,1.000000,False,
YXMUCHHM,700262462.0,1292008.0,harrys muenchen,harry s munchen,synxis,,48.1796,11.50684,48.179500,11.506132,53.046586,0.933333,0.928571,0.000000,1.000000,False,
HSDUSAIV,700125142.0,287918.0,sankt andreas garni,sankt andreas,hrs gds,,51.2269,6.77431,51.227007,6.774441,14.153666,0.812500,1.000000,0.000000,1.000000,True,
YXLEDRBB,700074651.0,40739.0,rinaldi petrogradskaya,rinaldi petrogradskaya side,synxis,,59.9633,30.30765,59.963282,30.306795,47.609014,0.897959,1.000000,0.000000,1.000000,True,
HSZSOACC,700029896.0,67447.0,werrapark haus frankenblick,werrapark frankenblick,hrs gds,,50.4993,10.90107,50.499324,10.900873,14.048632,0.897959,1.000000,0.000000,1.000000,False,
HSBJSBIZ,700127717.0,535607.0,broadcasting tower,beijing broadcasting tower,hrs gds,,39.9074,116.44191,39.907368,116.443335,121.715344,0.818182,1.000000,0.000000,1.000000,True,
HSZPRAAH,700088131.0,76935.0,floetzinger braeu,gasthof flotzinger brau,hrs gds,,47.8575,12.12823,47.857278,12.128084,22.961097,0.750000,0.937500,0.000000,1.000000,False,
YXCHS857,700081907.0,58452.0,seaside inn,seaside inn isle of palms,synxis,,32.7845,-79.78975,32.784562,-79.790414,62.194719,0.611111,1.000000,0.000000,1.000000,True,
WVNAP009,700038556.0,86982.0,club due torri maioni,club due torri,travelclick,,40.6802,14.62695,40.644178,14.649024,4418.111669,0.800000,1.000000,0.000000,1.000000,True,
HSCEKABA,700169056.0,308814.0,solnechny co heh,solnechniy,hrs gds,,55.148,61.36295,55.148082,61.363244,21.829033,0.692308,0.947368,0.000000,1.000000,False,


In [59]:
suspicious.to_excel('suspicious_matches_excel.xls', index=False)

# 5. remove suspicious matches and save to public matches file

In [60]:
len(matches)

85805

In [61]:
clean_matches = {k:v for k,v in matches.items() if k not in suspicious.amd_id.values}

len(clean_matches)

77314

In [62]:
# percentage matched
len(clean_matches) * 1.0 / len(amdh)

0.5494602335316149

In [63]:
json_dump_unicode(clean_matches,'matches_excel_clean.json')

# 6. Compute livingscores and append columns to excel file

In [66]:
amdh_full = pd.read_excel('./Amadeus All Properties - FEB 2017 17022017 .xlsx', header=0, index_col='PROPERTY_CODE', skiprows=1)

In [69]:
m = clean_matches

## Load livingscores for all Booking.com properties

In [64]:
with open('livingscore.json') as f:
    lvs = json.load(f)

## Map Livingrooms IDs to Amadeus codes

In [68]:
conn = get_amd_db_conn()
cur = conn.cursor()
query = """SELECT pr.id, pr.position, pr.name, v."vendorId"
           FROM public."Properties" pr
           JOIN public."VendorIds" v
           ON pr.id = v.relation
           WHERE v.vendor = 'amadeus'
        """
cur.execute(query)
lrid_to_amdid = {}
for r in cur.fetchall():
    lrid_to_amdid[r[0]] = r[-1]

amdid_to_lrid = {k:v for v,k in lrid_to_amdid.items()}

In [77]:
m.items()[0]

(u'700151432.0', 350152)

In [74]:
amdid_to_lrid.items()[0]

('YOFAYD02', '0afb7c67-406a-42ff-bfe1-e340df973eed')

In [78]:
amdh_full['DUPE_POOL_ID'].values

array([  5.03919399e+08,   5.03948101e+08,   5.03929932e+08, ...,
         7.00144748e+08,              nan,              nan])

In [ ]:
matches_ls = {}
for did, bkgid in m.items():
    matches_ls[did] = lvs.get(str(bkgid))
matches_ls = {k:v for k,v in matches_ls.items() if v}

def max_ls(did):
    ls = matches_ls.get(str(did))
    if ls:
        return max(ls)
    else:
        return None

def min_ls(did):
    ls = matches_ls.get(str(did))
    if ls:
        return min(ls)
    else:
        return None

def avg_ls(did):
    ls = matches_ls.get(str(did))
    if ls:
        return np.mean(ls)
    else:
        return None

amdh_full['LIVINGSCORE_MAX'] = amdh_full['DUPE_POOL_ID'].apply(max_ls)
amdh_full['LIVINGSCORE_MIN'] = amdh_full['DUPE_POOL_ID'].apply(min_ls)
amdh_full['LIVINGSCORE_AVG'] = amdh_full['DUPE_POOL_ID'].apply(avg_ls)

In [84]:
def did_to_bkg(did):
    return m.get(str(did))

amdh_full['Booking ID'] = amdh_full['DUPE_POOL_ID'].apply(did_to_bkg)

In [86]:
amdh_full.to_excel('Amadeus All Properties - FEB 2017 17022017 with Livingscores .xlsx')